# Векторная модель документа и TF-IDF

Еще этот метод называют метод <b>мешка слов</b> (<b>bag of words</b>).

Они хорошо работают, когда класс документа соответствует его тематике. Как правило, тематика документа хорошо описывается составом словаря, который используется в этом документе, а также частотами слов, а не тем, как именно они употребляются в документе, не структурой фраз. Тогда каждый документ описывается длинным вектором размерности порядка десятков или сотен тысяч элементов. Большая часть — нули. Для каждого слова в документе мы имеем какое-то вещественное число.

Очевидно, модели должны работать лучше, когда веса слов отличаются — чем значимее слово, тем больше его вес.

Рассмотрим несколько вариантов подсчета весов и разберем их приемущества и недостатки.

Самый простой вариант - взвесить слово по колличеству его употреблений в документе. Веса слов — это просто целые числа. Естественно, данный подход имеет недостатки. Во-первых, вес слова зависит от длины документа. В длинных документах слова имеют больший вес, как будто бы они более значимы, но это не так. Во-вторых, самые частотные слова — это союзы, предлоги, местоимения... Они встречаются везде, но абсолютно неинформативны и редко бывают полезны для каких-либо задач классификации.

Попробуем бороться с этими проблемами. Для начала отнормируем вектор документа на его длину.

$$n*w_{i} = \frac{w_{i}}{\sqrt{\sum_{j}w^2_{j}}}$$

Теперь веса слов будут зависеть от длинны документа гораздо слабее. Но они все равно будут зависеть, ведь с увеличением длины документа расширается словарный запас. Однако по прежнему предлоги и союзы это самые значимые слова

## TF-IDF

Самое время рассказать про один из фундаментальных эмпирических законов лингвистики (и не только лингвистики, на самом деле) — закон Ципфа. Возьмём большую коллекцию документов, посчитаем для каждого слова в этой коллекции частоту его встречаемости, то есть количество документов, в которых это слово используется, потом отсортируем полученный список по убыванию частот и получим примерно следующий график.

<img src="./Закон Ципфа.png">

По оси абсцисс отложим ранг слова, то есть его порядковый номер в отсортированном списке. По оси ординат отложим частоту слова — относительную частоту. Это график распределения Ципфа — распределения вероятностей, описывающего взаимоотношения частоты события и количества событий с такой частотой. Оно относится к классу степенных распределений и задаётся следующей функцей:

$$f(rank; s; N) = \frac{1}{Z(s,N)rank^s}$$

rank - порядковый номер слова после сортировки по убыванию частоты

s - коэффициент скорости убывания вероятности

N - количество слов

Z - нормализационная константа, чтобы распределение стало распределением

$$Z(s, N) = \sum^N_{i=1}i^{-s}$$

Из всего этого можно сделать вывод, что частотных слов мало и они неинформтивны
А вот редких слов очень много — если мы какое-нибудь редкое слово встречаем в документе, то мы с большой уверенностью можем сказать, к какой тематике он относится. Но проблема в том, что такие слова очень редки, и поэтому они ненадёжны в качестве факторов при принятии решений.

Значит, нам нужно придерживаться баланса частотности и информативности. 

Баланс частотности и информативности:
<ul>
    <li>Чаще встречается в документе - более характерен для этого документа</li>
    <li>Реже встречается в корпусе - более информативен</li>
</ul>

За этот баланс отвечают две величины: TF и IDF

TF (term frequency) — это частота слова в документе.
$$TF(w, d) = \frac{WordCount(w, d)}{Length(d)}$$

WordCount(w, d) - кол-во употреблений слова ц в документе d

Length(d) - длина документа d в словах

 IDF (inverse document frequency) — обратная частота слова в документах.
 $$IDF(w, c) = \frac{Size(c)}{DocCount(w, c)}$$

DocCount(w, c) - количество документов в коллекции с, в которых встречается слово w

Size(c) - размер коллекции с в документах



Тогда итоговый вес слова можно посчитать, как произведение этих двух величин
$$TFIDF(w, d, c) = TF(w, d) * IDF(w, c)$$

На практике, TF часто логарифмируют. Это позволяет сделать распределение весов слов менее контрастным и уменьшить его дисперсию. Логарифмирование поверх TF полезно, когда в Вашем датасете документы сильно отличаются по длине. В целом логарифмировать IDF тоже можно.

## Алгоритм взвешивания признаков по TF-IDF

<ol>
    <li>Применить нормализацию текста (стемминг или лемматизацию), выделить базовые элементы (символы, токены, n-граммы)</li>
    <li>Построить частотный словарь DocCount(w, c) для всех w</li>
    <li>Проредить словарь по частоте (выкинуть слишком редкие и слишком частые элементы)</li>
    <li>
        Для каждого документа d:
        <ol>
            <li>
                Для каждого слова w из d найти WordCount(w, d). <br>
                Записать в результирующий вектор в позицию w значение TFIDF (по формуле)
            </li>
            <li>Записать вектор документа в таблицу признаков документов коллекции</li>
        </ol>
    </li>
</ol>

2* проходимся по всей коллекции документов и, для каждого слова, подсчитываем, в каком количестве документов оно встретилось

3* Если у нас большая коллекция, то словарь может получиться просто гигантским, особенно если мы работаем с N-граммами, поэтому периодически, во время построения, или после этой процедуры, мы должны выбросить из словаря всё, что считаем неинформативным

4* Строим матрицу признаков. Каждая строчка этой матрицы соответствует документу, а каждый столбец — статистике встречаемости этого слова в документе

TF-IDF - это способ отбора категориальных признаков (не только в классификации и не только для текстов) 

TF-IDF никак не использует информацию о метке объекта — это одновременно и преимущество, и недостаток. Преимущество заключается в том, что мы можем использовать TF-IDF, не имея меток, то есть задачах обучения без учителя. Недостаток — в том, что мы теряем информацию или недостаточно эффективно её используем.

## Pointwise mutual inforamation (взаимная информация)

Еще один способ взвешивания по частоте - взаимная информация. Она измеряется между двумя случайными событиями или реализациями двух случайных величин. Она характеризует, насколько сильнее мы будем ожидать первое событие, если перед этим пронаблюдаем второе (по сравнению с нашими априорными ожиданиями). 

pmi(A, B):
<ul>
    <li>Измеряется между двумя случайными величинами A и B (или реализации двух случайных величин)</li>
    <li>Мера того, насколько сильнее мы будем ожидать появление А после наблюдения события B, по сравнению с нашими ожиданиями ситуации, когда событие B мы не наблюдали</li>
</ul>

Формулы для задачи классификации текстов:
$$pmi(I, w) = log\frac{p(w, I)}{p(w)P(I)} = log\frac{p(I|w)}{p(I)} = log\frac{p(w|I)}{p(w)}$$

I - коллекция документов, соответствующая некоторой метке класса L

w - слово из словаря

$$p(w, I) = \frac{DocCount(w, I)}{Size(I)}$$ - вероятность встретить слово w в документе класса L

$$p(w) = \frac{\sum_{I}DocCount(w, I)}{\sum_{I}Size(I)}$$ - маржинальная вероятность употребления слова w

$$p(I) = \frac{Size(I)}{\sum_{m}Size(m)}$$ - маржинальная вероятность встретить документ класса L 

Рассмотрим вот эту формулу: $$pmi(I, w) = log\frac{p(I|w)}{p(I)}$$  в знаменателе содержится уровень наших априорных ожиданий о появлении события L, а в числителе — уровень ожидания после наблюдения события W. Все три варианта формул на слайде эквивалентны.

Если возвращаться к текстам, то у нас есть два события. Первое — "L": "мы наблюдаем документ из класса L". Второе событие — "W": "мы видим в документе слово W". Все вероятности вычисляются по классическому определению вероятности, то есть как отношение количества положительных исходов к общему числу исходов.

Взаимная информация — это тоже способ взвешивания и отбора категориальных признаков. В первую очередь, он подходит для задач классификации. В задачах регрессии его тоже можно применять — например, дискретизировав целевое распределение, но это уже сложнее. Он требует наличия двух событий, что усложняет его применение в задачах обучения без учителя, хотя он используется для получения плотных векторных представлений слов.